In [1]:
from neuralnilm.source import RealApplianceSource, MultiSource, SameLocation
import matplotlib.pyplot as plt
import numpy as np
import logging

/home/dk3810/workspace/python/Lasagne/lasagne/init.py:86: UserWarning: The uniform initializer no longer uses Glorot et al.'s approach to determine the bounds, but defaults to the range (-0.01, 0.01) instead. Please use the new GlorotUniform initializer to get the old behavior. GlorotUniform is now the default for all layers.
  warnings.warn("The uniform initializer no longer uses Glorot et al.'s "


In [2]:
MAX_TARGET_POWER = 300

TARGET_APPLIANCE = ['fridge freezer', 'fridge', 'freezer']

SEQ_LENGTH = 512
TRAIN_BUILDINGS = [1, 2, 3, 4]
VALIDATION_BUILDINGS = [5]
N_SEQ_PER_BATCH = 8
SKIP_PROBABILITY_FOR_TARGET = 0.5

"""
WINDOW_PER_BUILDING = {
    1: ("2013-04-12", "2013-04-18"),
    2: ("2013-05-22", "2013-06-01"),
    3: ("2013-02-27", "2013-03-10"),
    4: ("2013-03-09", "2013-04-01"),
    5: ("2014-06-29", "2014-07-10")
}
"""

WINDOW_PER_BUILDING = {
    1: ("2012-12-15", "2014-12-01"),
    2: ("2013-05-22", "2013-10-01"),
    3: ("2013-02-27", "2013-04-01"),
    4: ("2013-03-09", "2013-09-20"),
    5: ("2014-06-29", "2014-08-27")
}


INPUT_STATS = {
    'mean': np.array([ 297.87216187], dtype=np.float32),
    'std': np.array([ 374.43884277], dtype=np.float32)
}

In [3]:
logger = logging.getLogger('test')
logger.setLevel(logging.DEBUG)

In [4]:
real_appliance_source1 = RealApplianceSource(
    logger=logger,
    filename='/data/mine/vadeec/merged/ukdale.h5',
    appliances=[
        TARGET_APPLIANCE,
        ['washer dryer', 'washing machine'],
        'dish washer',
        'kettle',
        'microwave'
    ],
    max_appliance_powers=[MAX_TARGET_POWER, 2400, 2500, 2600, 1500],
    on_power_thresholds=[5] * 5,
    min_on_durations=[60, 1800, 1800, 12, 12],
    min_off_durations=[12, 600, 1800, 12, 12],
    divide_input_by_max_input_power=False,
    window_per_building=WINDOW_PER_BUILDING,
    seq_length=SEQ_LENGTH,
    output_one_appliance=True,
    train_buildings=TRAIN_BUILDINGS,
    validation_buildings=VALIDATION_BUILDINGS,
    n_seq_per_batch=N_SEQ_PER_BATCH,
    skip_probability=0.75,
    skip_probability_for_first_appliance=SKIP_PROBABILITY_FOR_TARGET,
    target_is_start_and_end_and_mean=True,
    standardise_input=True,
    input_stats=INPUT_STATS
)

INFO:test:Loading training activations...
INFO:test:Setting window for building 1 to (start=2012-12-15, end=2014-12-01)
INFO:test:  Loading activations for fridge freezer from building 1...
INFO:test:    Loaded 18497 activations.
INFO:test:  Loading activations for washer dryer from building 1...
INFO:test:    Loaded 561 activations.
INFO:test:  Loading activations for dish washer from building 1...
INFO:test:    Loaded 226 activations.
INFO:test:  Loading activations for kettle from building 1...
INFO:test:    Loaded 3355 activations.
INFO:test:  Loading activations for microwave from building 1...
INFO:test:    Loaded 3929 activations.
INFO:test:Setting window for building 2 to (start=2013-05-22, end=2013-10-01)
INFO:test:  Loading activations for fridge freezer from building 2...
INFO:test:    Loaded 4 activations.
INFO:test:  Loading activations for washer dryer from building 2...
INFO:test:    Loaded 50 activations.
INFO:test:  Loading activations for dish washer from building 2..

In [5]:
same_location_source1 = SameLocation(
    logger=logger,
    filename='/data/mine/vadeec/merged/ukdale.h5',
    target_appliance=TARGET_APPLIANCE,
    window_per_building=WINDOW_PER_BUILDING,
    seq_length=SEQ_LENGTH,
    train_buildings=TRAIN_BUILDINGS,
    validation_buildings=VALIDATION_BUILDINGS,
    n_seq_per_batch=N_SEQ_PER_BATCH,
    skip_probability=SKIP_PROBABILITY_FOR_TARGET,
    target_is_start_and_end_and_mean=True,
    standardise_input=True,
    offset_probability=1,
    divide_target_by=MAX_TARGET_POWER,
    input_stats=INPUT_STATS
)

INFO:test:Setting window for building 1 to (start=2012-12-15, end=2014-12-01)
INFO:test:Loaded 18307 fridge freezer activations from house 1.
INFO:test:Setting window for building 2 to (start=2013-05-22, end=2013-10-01)
INFO:test:Loaded 3436 fridge activations from house 2.
INFO:test:Setting window for building 3 to (start=2013-02-27, end=2013-04-01)
INFO:test:Building 3 has no ['fridge freezer', 'fridge', 'freezer']
INFO:test:Removing building 3 from validation_buildings and train_buildings.
INFO:test:Setting window for building 4 to (start=2013-03-09, end=2013-09-20)
INFO:test:Loaded 4592 freezer activations from house 4.
INFO:test:Setting window for building 5 to (start=2014-06-29, end=2014-08-27)
INFO:test:Loaded 1376 fridge freezer activations from house 5.
INFO:test:Loading mains data for building 1...
INFO:test:  Setting window for building 1 to (start=2012-12-15, end=2014-12-01)
INFO:test:  Loaded mains data for building 1.
INFO:test:Mains starts after activation 1726 so will r

In [6]:
multi_source = MultiSource(
    sources=[
        {
            'source': real_appliance_source1,
            'train_probability': 0.5,
            'validation_probability': 0
        },
        {
            'source': same_location_source1,
            'train_probability': 0.5,
            'validation_probability': 1
        }
    ],
    standardisation_source=same_location_source1
)

In [12]:
batch = multi_source.get_batch()
#same_location_source1.skip_probability=0
#batch = same_location_source1.get_batch(validation=True)
#batch = real_appliance_source1.get_batch(validation=True)
X, y = batch.data
print(X.shape)
print(y.shape)
print(y[0, :, 0])
print("source_i =", batch.source_i)
fig, axes = plt.subplots(2, sharex=True)
axes[0].plot(X[0, :, 0])
axes[1].plot(batch.target_power_timeseries[0, :, 0])
plt.show()

(8, 512, 1)
(8, 3, 1)
[ 0.54882812  0.92382812  0.29644096]
source_i = 0
